In [3]:
from IPython.display import display, HTML


Import DS modules

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import statsmodels.api as sm

## Getting Started

In this notebook, we will walk through what it takes to build and assess quality of a simple strategy

We will rely heavily on `TDSCoinbaseData`, `TDSTickGenerator`, and `TDSTransactionTracker`

In [5]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

In [6]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='data', notebook_logging=True)

In [7]:
# Set the start and end date
start_date = '20201001'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR', 'LTC-USD']

In [8]:
# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

In [9]:
# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)

## Init Pandas Dataframe aka timeseries in this setting.

In [10]:
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
# history dataframes
btc_usd = pd.DataFrame(data=d)
eth_usd = pd.DataFrame(data=d)
ltc_usd = pd.DataFrame(data=d)
#bch_usd = pd.DataFrame(data=d)
# prediction
pred_btc = None
pred_eth = None
pred_ltc = None
# TODO: The flag for normal/low/high volatility period

starttime = '2020-01-01 00:00:00+00:00'
holding_btc = True;
size = 0
MAX_TRAINNING_SIZE = 1440
SIZE_LIMIT_REACHED = False

## Loads Initial data (for testing only)
## The model (probably) doesn't work on small sample size


In [11]:
# Appends value for a new tick to existing dataframes
# Input: 
#    tick -- the tick obj
#    btc_usd -- dataframe
#    eth_usd -- dataframe
#    scale_btc -- the scale value for btc/dollar
#    eth_btc -- the scale value for eth/dollar
def append_ticks(tick, btc_usd, eth_usd, ltc_usd, 
                 max_r=SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=100000, scale_ltc=100000):
    t_usd = tick.p.btc_usd
    t_eth = tick.p.eth_usd
    t_ltc = tick.p.ltc_usd
    btc_usd = btc_usd.append({'open': scale_btc/t_usd.open, 'close': scale_btc/t_usd.close, 
                      'volume': t_usd.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_usd = eth_usd.append({'open': scale_eth/t_eth.open, 'close': scale_eth/t_eth.close, 
                      'volume': t_eth.volume, 'datetime': tick.datetime}, ignore_index=True)
    ltc_usd = ltc_usd.append({'open': scale_ltc/t_ltc.open, 'close': scale_ltc/t_ltc.close, 
                      'volume': t_ltc.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(btc_usd.index) > MAX_TRAINNING_SIZE) :
        btc_usd.iloc[1:]
        eth_usd.iloc[1:]
        ltc_usd.iloc[1:]
        max_r = True
    return(btc_usd, eth_usd, ltc_usd, max_r)

#loads initial value for testing
for i in range(0, 720) :
    tick = tick_gen.get_tick()
    (btc_usd, eth_usd, ltc_usd, SIZE_LIMIT_REACHED) = append_ticks(tick, btc_usd, eth_usd, ltc_usd)


In [12]:
ltc_usd

,open,close,volume,datetime
0,2158.428664,2160.760588,15.458457,2020-10-01 00:00:00+00:00
1,2160.760588,2160.760588,0.000000,2020-10-01 00:01:00+00:00
2,2160.293800,2160.293800,21.576887,2020-10-01 00:02:00+00:00
3,2159.360829,2161.227577,25.363349,2020-10-01 00:03:00+00:00
4,2159.827214,2157.031924,17.946333,2020-10-01 00:04:00+00:00
...,...,...,...,...
715,2087.682672,2087.682672,4.135373,2020-10-01 11:55:00+00:00
716,2087.682672,2087.682672,0.000000,2020-10-01 11:56:00+00:00
717,2088.118605,2088.118605,136.021525,2020-10-01 11:57:00+00:00
718,2087.682672,2087.682672,13.154790,2020-10-01 11:58:00+00:00


In [15]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = frame.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1, 0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
    #print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[-1]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
    #print(pred.predicted_mean)
#     print(frame.iloc[-1])
    return (pred)

# Call this function for btc prediction
#     minute -- the number of minutes to be predicted
def predict_btc(minute) :
    return predict(btc_usd, minute)
    
def predict_eth(minute) :
    return predict(eth_usd, minute)

def predict_ltc(minute) :
    return predict(ltc_usd, minute)


## This part below will be the actual while loop
(while is commented out at this point)

In [ ]:
import random
hr = 0

## For the rest of the trading period we will randomly flip our BTC and USD holdings
while tick is not None:
    print("Time:", tick.datetime.to_pydatetime().hour)
    print("holdings", trans_tracker.get_holdings())
#     print("Time:", tick.datetime.dt.hour)
#     print("holdings", trans_tracker.get_holdings())
    if (tick and not hr == 0 and hr != tick.datetime.to_pydatetime().hour):
        print("Time:", tick.datetime.to_pydatetime().hour)
        print("holdings", trans_tracker.get_holdings())
        hr = tick.datetime.to_pydatetime().hour
        # 10% chance of attempting to trade
    tick = tick_gen.get_tick()
#         no trade for this tick
    if (not tick.p.btc_usd.volume and not tick.p.eth_usd.volume and not tick.p.ltc_usd.volume):
        continue
    (btc_usd, eth_usd, ltc_usd, SIZE_LIMIT_REACHED) = append_ticks(tick, btc_usd, eth_usd, ltc_usd, SIZE_LIMIT_REACHED)
    if random.uniform(0,1) <= 0.1:
        if holding_btc and random.uniform(0,1) >= 0.5:
            # Check the max amount of BTC we can trade my multiplying the volume by the max_taken_volume (0.5 in our case)
            available_btc = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume()
            my_btc = trans_tracker.get_holdings()['BTC']
            # Only trade if valid
            if available_btc >= my_btc and random.uniform(0,1) >= 0.3:
                trans_tracker.make_trade(tick, 'BTC-USD', 'sell', 0.1)
                holding_btc = False
        else:
            # Check the available USD by doing the same operation as above and multiplying by the close price since that is the price that we will execute at
            available_usd = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume() * tick.p.btc_usd.close
            my_usd = trans_tracker.get_holdings()['USD']
            # Only trade if valid
            if available_usd >= my_usd and random.uniform(0,1) >= 0.7:
                trans_tracker.make_trade(tick, 'BTC-USD', 'buy', 0.1)
                holding_btc = True

        # the int passed in here is the number of minutes about to predict
    pred_btc = predict_btc(3)
    #print(pred_usd.predicted_mean)
#     print(pred_btc.conf_int())


    # pred_usd_next = predict_btc(3)

    pred_eth = predict_eth(3)
    pred_ltc = predict_ltc(3)
#print(pred_eth.predicted_mean)
#print(pred_eth.conf_int())

Time: 13
holdings {'BTC': 0.600036685989014, 'USD': 4345.8396614}


/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information w

Time: 13
holdings {'BTC': 0.600036685989014, 'USD': 4345.8396614}


/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)


Time: 13
holdings {'BTC': 0.600036685989014, 'USD': 4345.8396614}


...

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimizat

Time: 14
holdings {'BTC': 0.40004587650203294, 'USD': 6513.2071846}


/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)


Time: 14
holdings {'BTC': 0.40004587650203294, 'USD': 6513.2071846}


/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)


In [383]:
pred_eth.predicted_mean

,open,close,volume
2020-10-01 12:02:00+00:00,271.347566,271.383743,102.787626
2020-10-01 12:03:00+00:00,271.621763,271.599044,35.447909
2020-10-01 12:04:00+00:00,271.602462,271.602323,66.040436
2020-10-01 12:05:00+00:00,271.605322,271.609218,69.830577


In [384]:
pred_eth.conf_int()

,lower open,lower close,lower volume,upper open,upper close,upper volume
2020-10-01 12:02:00+00:00,271.269856,271.046052,-168.826631,271.425276,271.721433,374.401884
2020-10-01 12:03:00+00:00,271.544053,271.261353,-236.166349,271.699473,271.936734,307.062167
2020-10-01 12:04:00+00:00,271.250707,271.138338,-213.741109,271.954216,272.066308,345.821980
2020-10-01 12:05:00+00:00,271.130842,271.048858,-210.056442,272.079801,272.169578,349.717595


In [ ]:
def difference(pred, expected):
    pred - 

Debug Mode

In [ ]:
rec = rec.set_index('datetime')
y = rec['open'].resample('1T').mean()
l = rec['close'].resample('1T').mean()
v = rec['volume'].resample('1T').mean()
merged = pd.merge(y, l, on='datetime', how="outer")
rec = pd.merge(merged, v, on='datetime', how="outer")

mod = sm.tsa.VARMAX(rec[['open', 'close', 'volume']], order=(1,0), error_cov_type='diagonal')
results = mod.fit(maxiter=50, disp=False)
print(results.summary().tables[1])
newtime = tick.datetime.to_pydatetime() + timedelta(minutes=3)
pred = results.get_prediction(start=pd.to_datetime('2020-10-01 00:22:00+00:00'), end=newtime, dynamic=False)
pred_ci = pred.conf_int()
print(pred.predicted_mean)

## Visualizing Our Returns

Once we have completed trading, we can visualize our returns by calling `plot_btc_holdings()`

In [19]:
trans_tracker.plot_btc_holdings()

KeyboardInterrupt: 

## Looking at Daily Stats

We can call `get_pct_change_per_day()` to get a dataframe of our daily `BTC` holdings and day to day percent change

`BTC` holding for a single day is derived by taking the shortest path through each product to `BTC` using the last transaction price of a given day

EX:
To get the `BTC` value of an `ETH` holding, we traverse `ETH` -> `USD` -> `BTC`

This logic is all handled for you as a part of `TDSTransactionTracker`

In [20]:
display(trans_tracker.get_pct_change_per_day())

,date,BTC,pct_diff
0,20201001,0.829108,-0.170892
1,20201002,0.702294,-0.152952
2,20201003,0.630000,-0.102939
3,20201004,0.537633,-0.146614
4,20201005,0.454940,-0.153811
...,...,...,...
87,20201227,0.093548,0.066443
88,20201228,0.095426,0.020067
89,20201229,0.094541,-0.009270
90,20201230,0.091634,-0.030746


## Compute Sharpe Ratio

To get out Sharpe ration, we simply call `get_sharpe_ratio()`

This Sharpe ratio is based on the EOD BTC holdings as shown above and uses the BlockFi 1 BTC interest rate as the risk free return

https://blockfi.com/crypto-interest-account

In [21]:
print(trans_tracker.get_sharpe_ratio())

RISK FREE DAILY RETURN : 
0.00016438356164383562
ACTUAL DAILY RETURN : 
-0.024572212328816654
EXCESS STD : 
0.049256698748459345
-0.5021975998997336


## Saving Your Results

To save your trades to a json file in the required format, when you are done trading, simply call `dump_trades("<filepath>")`

In [ ]:
trans_tracker.dump_trades('example_trades.json')